# Business Cycle Facts and Real Business Cycles Model

We will take a look at the data on business cycle fluctuations in the United States and then see if we can explain their behavior with the help of the basic RBC model.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pandas_datareader.data import DataReader

In [ ]:
from matplotlib import rcParams

# Restore old behavior of rounding default axis ranges
rcParams['axes.autolimit_mode'] = 'round_numbers'
rcParams['axes.xmargin'] = 0
rcParams['axes.ymargin'] = 0

# Adjust tick placement
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'
rcParams['xtick.top'] = True
rcParams['ytick.right'] = True

# Disable legend frame
rcParams['legend.frameon'] = False

Below we are downloading data from Federal Reserve Bank of St. Louis Economic Database (FRED) https://fred.stlouisfed.org/ and organizing them into three datasets.

The first dataset, `rec`, contains monthly data on whether in a given month the US economy was in a recession state (`1`) or not (`0`) - `USREC`

*Recession is a "business cycle contraction when there is a general decline in economic activity. Recessions generally occur when there is a widespread drop in spending (an adverse demand shock). This may be triggered by various events, such as a financial crisis, an external trade shock, an adverse supply shock, the bursting of an economic bubble, or a large-scale anthropogenic or natural disaster (e.g. a pandemic)."*

*In the United States, it is defined as "a significant decline in economic activity spread across the market, lasting more than a few months, normally visible in real GDP, real income, employment, industrial production, and wholesale-retail sales".*

*In the United Kingdom, it is defined as a negative economic growth for two consecutive quarters.*

The second dataset, `pop`, containts quarterly data on:
- total population in the US - `B230RC0Q173SBEA`
- civilian noninstitutional population of 16 years of age and older - `CNP16OV`

*Civilian noninstitutional population is defined as persons 16 years of age and older residing in the 50 states and 
the District of Columbia, who are not inmates of institutions (e.g., penal and mental facilities, homes for the aged), 
and who are not on active duty in the Armed Forces.*

The third dataset, `fred`, contains quarterly data on:

- nominal Gross Domestic Product in billions of current dollars - `GDP`
- nominal Personal Consumption Expenditures on Nondurable Goods - `PCND`
- nominal Personal Consumption Expenditures on Services - `PCESV`
- nominal Personal Consumption Expenditures on Durable Goods - `PCDG`
- nominal Fixed Private Investment - `FPI`
- Implicit Price Deflator for Gross Domestic Product - `GDPDEF`
- Nonfarm Business Sector: Current Dollar Output - `PRS85006053`
- total hours worked in the nonfarm business sector (index) - `HOANBS`
- nominal compensation per hour in the nonfarm business sector (index) - `COMPNFB`
- interest rate on 3-month treasury bill - `TB3MS`

In [ ]:
start = '1945-01'
end   = '2021-06'

In [ ]:
rec = DataReader('USREC', 'fred', start=start, end=end)
rec = rec.dropna()
rec.tail()



In [ ]:
pop = DataReader(['B230RC0Q173SBEA', 'CNP16OV'], 'fred', start=start, end=end)
pop = pop.dropna()
pop = pop.resample('QS').mean()
pop = pop.dropna()

pop.tail()

In [ ]:
fred = DataReader(['GDP', 'PCND', 'PCESV', 'PCDG', 'FPI', 'GDPDEF',
                   'PRS85006053', 'HOANBS', 'COMPNFB', 'TB3MS'],
                  'fred', start=start, end=end)
fred = fred.dropna()
fred = fred.resample('QS').mean()
fred = fred.dropna()
fred.head()

In [ ]:
fred.tail()

Since the population data are based on the population census, they exhibit periodic "jumps" that we will filter out. 
The time series are composed of three components: seasonal, cyclical, and trend. Assume that we have seasonally adjusted time series and we want to decompose time series into trend and cyclical components.<br> 
The Hodrick–Prescott filter (also known as Hodrick–Prescott decomposition) is a tool to remove the cyclical component of a time series from raw data.<br> 
Let $y_{t}$, for $t=1,2,...,T$ denote the logarithms of a time series variable. The series $y_{t}$, is made up of a trend component $\tau_{t}$, and a cyclical component $c_{t}$ such that $y_{t}=\tau_{t}+c_{t}$. Given an adequately chosen, positive value of $\lambda$ , the filter will decompose $y_{t}$ into $\tau_{t}$ and $c_{t}$.<br> 
$\lambda$ is the Hodrick-Prescott smoothing parameter. A value of $1600$ is suggested for quarterly data. Ravn and Uhlig suggest using a value of $6.25$ for annual data and $129600$ for monthly data.

In [ ]:
pop_hp_cycle = pd.DataFrame()
pop_hp_trend = pd.DataFrame()

for col in pop.columns:
    pop_hp_cycle[col], pop_hp_trend[col] = sm.tsa.filters.hpfilter(np.log(pop[col]), lamb=1600)
    
pop_smooth = np.exp(pop_hp_trend)

In [ ]:
fig, ax = plt.subplots()

pop['CNP16OV'].to_period('D').plot(ax=ax, lw=2)
pop_smooth['CNP16OV'].to_period('D').plot(ax=ax, lw=2)
plt.legend(['raw', 'filtered'])
plt.show()

fig, ax = plt.subplots()

(100*pop['CNP16OV'].pct_change()).to_period('D').plot(ax=ax, lw=2)
(100*pop_smooth['CNP16OV'].pct_change()).to_period('D').plot(ax=ax, lw=2)
plt.legend(['raw', 'filtered'], loc='upper left')
plt.show()

Below we are constructing the series that will correspond to model objects. All variables will be expressed in logarithms:
- `Output` is real GDP per person (aged 16 and over)
- `Consumption` is the real sum of expenditures on nondurable goods and services per person
- `Investment` is the real sum of expenditures on durable goods and fixed private investment per person
- `Capital` will contain real capital stock per person (to be constructed later)
- `Hours` is hours worked per person, corrected for the difference between the entire economy and the nonfarm business sector
- `Wages` is real compensation per hour worked
- `Interest Rate` is the real quarterly return on the 3 month treasury bill
- `TFP` will contain Total Factor Productivity series (to be constructed later)
- `Productivity` is the real output per hour worked in the nonfarm business sector
- `Price Level` is the GDP implicit price deflator

In [ ]:
dta = pd.DataFrame()

dta['Output'] = np.log(fred['GDP']*10**9/fred['GDPDEF']*100
                       /(pop_smooth['CNP16OV']*10**3))

dta['Consumption'] = np.log((fred['PCND']+fred['PCESV'])*10**9/fred['GDPDEF']*100
                            /(pop_smooth['CNP16OV']*10**3))

dta['Investment'] = np.log((fred['PCDG']+fred['FPI'])*10**9/fred['GDPDEF']*100
                           /(pop_smooth['CNP16OV']*10**3))

dta['Capital'] = 0*dta['Output']

dta['Hours'] = np.log(fred['HOANBS']*100*fred['GDP']/np.mean(fred['GDP']['2010-01':'2010-10'])
                      /fred['PRS85006053']/pop_smooth['CNP16OV'])

dta['Wages'] = np.log(fred['COMPNFB']/fred['GDPDEF']*100)

dta['Interest Rate'] = np.log( (1+fred['TB3MS']/100)**(1/4)/(1+fred['GDPDEF'].pct_change()) )

dta['TFP'] = 0*dta['Output']

dta['Productivity'] = np.log(fred['PRS85006053']/fred['GDPDEF']*100)-dta['Hours']

dta['Price Level'] = np.log(fred['GDPDEF'])

dta = dta.dropna()

dta.head()

In [ ]:
dta.tail()

Below we will estimate the capital stock based on the Perpetual Inventory Method (PIM) that simply applies the capital accumulation equation:

\begin{align}
K_{t+1} = I_{t} + \left( 1-\delta \right) K_{t}
\end{align}

We need to "start" this procedure from a certain level of initial capital stock, $K_{0}$, which is below calculated basing on the investment trend in the beginning of the sample.

In [ ]:
# Estimate Capital series using PIM
temp = pd.DataFrame()

temp['Inv'] = (fred['PCDG']+fred['FPI'])/fred['GDPDEF']*100/4
temp['LnInv'] = np.log(temp['Inv'])
temp['t'] = np.arange(len(temp['Inv']))

trend = smf.ols(formula='LnInv ~ t', data=temp).fit()
intercept, slope = trend.params
print(trend.summary())


delta = 0.025
K = np.zeros(len(temp['LnInv']))
K_init = np.exp(intercept-slope)/(slope+delta)

K[0] = (1-delta)*K_init+temp['Inv'][0]
for i in range(1,len(temp['Inv'])):
    K[i] = (1-delta)*K[i-1]+temp['Inv'][i]
temp['Cap'] = K

temp['Cap'].to_period('D').plot(lw=2)
plt.title('Capital stock')
plt.show()

(temp['Cap']/(fred['GDP']/fred['GDPDEF']*100)).to_period('D').plot(lw=2)
plt.title('$K / Y$')
plt.show()

We recover the constructed time series for capital and construct the TFP measure from the production function:

\begin{align}
Y_{t} &= K_{t}^{\alpha} \left( A_{t} L_{t} \right)^{1-\alpha} \\
Y_{t} &= TFP_{t} \cdot K_{t}^{\alpha} L_{t}^{1-\alpha} \\
TFP_{t} &= Y_{t} / \left( K_{t}^{\alpha} L_{t}^{1-\alpha} \right) \\
\log TFP_{t} &= \log Y_{t} - \alpha \log K_{t} - \left(1-\alpha \right) \log L_{t}
\end{align}

In [ ]:
dta['Capital'] = np.log(temp['Cap']*10**9/(pop_smooth['CNP16OV']*10**3))

α = 1/3
dta['TFP'] = dta['Output']-α*dta['Capital']-(1-α)*dta['Hours']
dta['TFP'].to_period('D').plot(lw=2)
plt.title('Total Factor Productivity (log)')
plt.show()

In [ ]:
RGDP_pc = (fred['GDP']*10**9/fred['GDPDEF']*100/(pop_smooth['B230RC0Q173SBEA']*10**3)).dropna()

fig, ax = plt.subplots()

RGDP_pc.to_period('D').plot(ax=ax, lw=2, style='k-')

ax.set_ylim(10000, 70000)
ylim = ax.get_ylim()

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP (2009 dollars)')
plt.show()

In [ ]:
fig, ax = plt.subplots()

np.log(RGDP_pc).to_period('D').plot(ax=ax, lw=2, style='k-')

ticks = [10000, 20000, 30000, 40000, 50000, 60000, 70000]
ax.set_yticks(np.log(ticks))
ax.set_yticklabels(ticks)

ax.set_ylim(np.log(ticks[0]), np.log(ticks[-1]))

ylim = ax.get_ylim()
ax.set_ylim(ylim)

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, log scale')
plt.show()

In [ ]:
fig, ax = plt.subplots()

(100*RGDP_pc.pct_change(4)).to_period('D').plot(ax=ax, lw=2, style='k-')

avg = np.mean(100*RGDP_pc.to_period('D').pct_change(4))

# ax.set_ylim(-6, 12)
ylim = ax.get_ylim()

ax.hlines(avg, dta.index[0], dta.index[-1], color='r', linewidth=2)
ax.hlines(0, dta.index[0], dta.index[-1], linewidth=0.5)

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, year-over-year change (%)')
plt.show()

print('Average growth rate (%) =', round(avg,2))

In [ ]:
gdp_lin_cycle, gdp_lin_trend = sm.tsa.filters.hpfilter(np.log(RGDP_pc), lamb=1e9)
gdp_hp_cycle, gdp_hp_trend = sm.tsa.filters.hpfilter(np.log(RGDP_pc), lamb=1600)
gdp_cf_cycle, gdp_cf_trend = sm.tsa.filters.cffilter(np.log(RGDP_pc), low=6, high=32)

In [ ]:
fig, ax = plt.subplots()

np.log(RGDP_pc).to_period('D').plot(ax=ax, lw=2, style='k-')
gdp_lin_trend.plot(ax=ax, lw=2, style='r-')

ticks = [10000, 20000, 30000, 40000, 50000, 60000, 70000]
ax.set_yticks(np.log(ticks))
ax.set_yticklabels(ticks)

ax.set_ylim(np.log(ticks[0]), np.log(ticks[-1]))

ylim = ax.get_ylim()
ax.set_ylim(ylim)

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, linear trend')
plt.show()

In [ ]:
fig, ax = plt.subplots()

(100*gdp_lin_cycle).to_period('D').plot(ax=ax, lw=2, style='k-')
ax.hlines(0, dta.index[0], dta.index[-1], linewidth=0.5)

ylim = ax.get_ylim()
ax.set_ylim(ylim)

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, linear trend residual (%)')
plt.show()

In [ ]:
fig, ax = plt.subplots()

np.log(RGDP_pc).to_period('D').plot(ax=ax, lw=2, style='k-')
gdp_hp_trend.plot(ax=ax, lw=2, style='r-')

ticks = [10000, 20000, 30000, 40000, 50000, 60000, 70000]
ax.set_yticks(np.log(ticks))
ax.set_yticklabels(ticks)

ax.set_ylim(np.log(ticks[0]), np.log(ticks[-1]))

ylim = ax.get_ylim()
ax.set_ylim(ylim)

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, Hodrick-Prescott trend')
plt.show()

In [ ]:
fig, ax = plt.subplots()

(100*gdp_hp_cycle.to_period('D')).plot(ax=ax, lw=2, style='k-')
ax.hlines(0, dta.index[0], dta.index[-1], linewidth=0.5)

ax.set_ylim(-6, 6)
ylim = ax.get_ylim()

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, Hodrick-Prescott residual (%)')
plt.show()

In [ ]:
fig, ax = plt.subplots()

(100*gdp_cf_cycle).to_period('D').plot(ax=ax, lw=2, style='k-')
ax.hlines(0, dta.index[0], dta.index[-1], linewidth=0.5)

ylim = ax.get_ylim()

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.title('US real per capita GDP, Christiano-Fitzgerald residual (%)')
plt.show()

In [ ]:
np.corrcoef(gdp_hp_cycle, gdp_cf_cycle)

In [ ]:
hp_cycles = pd.DataFrame()
hp_trend = pd.DataFrame()

cf_cycles = pd.DataFrame()
cf_trend = pd.DataFrame()

for col in dta.columns:
    hp_cycles[col], hp_trend[col] = sm.tsa.filters.hpfilter((100*dta[col]).dropna(), lamb=1600)
    cf_cycles[col], cf_trend[col] = sm.tsa.filters.cffilter((100*dta[col]).dropna(), low=6, high=32)

In [ ]:
print('Standard Deviations')
print(hp_cycles.std())

print('')
print('Autocorrelations')
a = list(dta.columns.values)
for i in range(len(a)):
    print(dta.columns.values[i], '  \t\t', hp_cycles[dta.columns.values[i]].autocorr())

print('')
print('Correlations')
print(hp_cycles.corr(method='pearson'))

In [ ]:
cf_cycles['Investment / 4'] = cf_cycles['Investment'] / 4

fig, axs = plt.subplots(3, 3, figsize=(20, 15), sharex=False, sharey=False)

cf_cycles[['Output','Consumption']].to_period('D').plot(ax=axs[0, 0], style=['k','r'], lw=2)

cf_cycles[['Output','Investment']].to_period('D').plot(ax=axs[0, 1], style=['k','r'], lw=2)

cf_cycles[['Output','Capital']].to_period('D').plot(ax=axs[0, 2], style=['k','r'], lw=2)

cf_cycles[['Output','Hours']].to_period('D').plot(ax=axs[1, 0], style=['k','r'], lw=2)

cf_cycles[['Output','Wages']].to_period('D').plot(ax=axs[1, 1], style=['k','r'], lw=2)

cf_cycles[['Output','Interest Rate']].to_period('D').plot(ax=axs[1, 2], style=['k','r'], lw=2)

cf_cycles[['Output','TFP']].to_period('D').plot(ax=axs[2, 0], style=['k','r'], lw=2)

cf_cycles[['Output','Productivity']].to_period('D').plot(ax=axs[2, 1], style=['k','r'], lw=2)

cf_cycles[['Output','Price Level']].to_period('D').plot(ax=axs[2, 2], style=['k','r'], lw=2)

for ax_a in axs:
    for ax in ax_a:
        ax.set_xlabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots()

cf_cycles[['Output','Consumption','Investment / 4','Hours']].to_period('D').plot(ax=ax, lw=2)

ylim = ax.get_ylim()

ax.fill_between(rec.index, ylim[0], ylim[1], rec['USREC'], facecolor='lightgrey', edgecolor='lightgrey')

plt.legend(ncol=2, frameon=True)
plt.show()

In [ ]:
dta['t'] = np.arange(len(dta['TFP']))
trend_TFP = smf.ols(formula='TFP ~ t', data=dta).fit()
dta['TFP_resid'] = trend_TFP.resid
intercept_TFP, slope_TFP = trend_TFP.params
print(trend_TFP.summary())

In [ ]:
dta['TFP'].to_period('D').plot(lw=2)
(intercept_TFP + slope_TFP*dta['t']).plot(lw=2)
plt.show()

In [ ]:
dta['TFP_resid'].to_period('D').plot(lw=2)
plt.show()

In [ ]:
resid_TFP = smf.ols(formula='TFP_resid ~ TFP_resid.shift() -1', data=dta).fit()
print(resid_TFP.summary())

print('')
print('TFP residual autocorrelation    =', resid_TFP.params[0])
print('TFP residual standard deviation =', resid_TFP.resid.std())

In [ ]:
rho_z    = resid_TFP.params[0]
stderr_z = resid_TFP.resid.std()

#  Real Business Cycles (RBC) models 

Dynamic Stochastic General Equilibrium (DSGE) models aimto replicate business cycle behavior of real-world economies

– Dynamic: forward-looking behavior of agents

– Stochastic: the economy is subject to shocks

– GE: what happens in one market influences other markets\smallskip

We can generate quantitative predictions on short-term movements of macro variables and compare them with the data\medskip

We use those models to

– Simulate counterfactual scenarios

– Explain past developments (historical decomposition)

– Construct forecasts (conditional and uncoditional)

– Perform policy experiments\smallskip

## The RBC Model
Ramsey model with endogenous labor supplyand stochastic “technology” shocks<br><br>
Closed economy with no government<br><br>
Perfect competition<br><br>
Single final good with price normalized to 1 – all other prices are real<br><br>
Two groups of representative agents

– Households

– Firms\smallskip

Rational expectations – agents make no systematic forecast errors

### Household

A representative household solves the expected utility maximization problem:

\begin{align}
\max_{c_{t},a_{t+1}}\quad & U_{t}=E_{t}\left[\sum_{t=0}^{\infty}\beta^{t}\left(\log c_{t}-\phi \log \left(1-h_{t}\right)\right)\right]\\
\text{subject to}\quad & c_{t}+a_{t+1}=\left(1+r_{t}\right)a_{t}+w_{t}h_{t}+d_{t}
\end{align}

where:

$\beta$	- discount factor<br>
$c$	- per capita consumption<br>
$\phi$	- relative preference for leisure<br>
$h$	- per capita hours (as fraction of total available time)<br>
$a$	- per capita assets (physical capital)<br>
$r$	- real interest rate<br>
$w$	- real wage per hour<br>
$d$	- per capita dividends

Write down the Lagrangian:
\begin{align}
\mathcal{L}	=\sum_{t=0}^{\infty}\beta^{t}\mathrm{E}_{t}\left[\ln c_{t}+\phi\ln\left(1-h_{t}\right)\right]+\sum_{t=0}^{\infty}\beta^{t}\mathrm{E}_{t}\left[\lambda_{t}\left[\left(1+r_{t}\right)a_{t}+w_{t}h_{t}+d_{t}-a_{t+1}-c_{t}\right]\right]
\end{align}

FOCs:
\begin{align}
\frac{\partial\mathcal{L}}{\partial c_{t}} &	=  \mathrm{E}_{t}\left[\frac{1}{c_{t}}\right]-\mathrm{E}_{t}\left[\lambda_{t}\right]=0\to\quad & \lambda_{t} & =\frac{1}{c_{t}}\\
\frac{\partial\mathcal{L}}{\partial h_{t}} &	=  \mathrm{E}_{t}\left[-\frac{\phi}{1-h_{t}}\right]+\mathrm{E}_{t}\left[\lambda_{t}w_{t}\right]=0\to\quad & \lambda_{t} & =\frac{\phi}{w_{t}\left(1-h_{t}\right)}\\
\frac{\partial\mathcal{L}}{\partial a_{t+1}} &	=  - \mathrm{E}_{t}\left[\lambda_{t}\right]+\beta\mathrm{E}_{t}\left[\lambda_{t+1}\left(1+r_{t+1}\right)\right]=0
	\to\quad & \lambda_{t} & =\beta\mathrm{E}_{t}\left[\lambda_{t+1}\left(1+r_{t+1}\right)\right]
\end{align}

Optimality conditions:
\begin{align}
\text{Intertemporal condition }\left(c+a\right)	& :	\frac{1}{c_{t}}=\beta\mathrm{E}_{t}\left[\frac{1}{c_{t+1}}\left(1+r_{t+1}\right)\right]\\
\text{Intratemporal condition }\left(c+h\right)	& :	\frac{1}{c_{t}} =\frac{\phi}{w_{t}\left(1-h_{t}\right)}\to\quad h_{t}=1-\phi\frac{c_{t}}{w_{t}}
\end{align}


### Firms
A representative firm solves profit (dividend) maximization problem:

\begin{align}
\max\quad&d_{t}=y_{t}-r_{t}^{k}k_{t}-w_{t}h_{t}\\\text{subject to}\quad&y_{t}=z_{t}k_{t}^{\alpha}h_{t}^{1-\alpha}\\&r_{t}^{k}=r_{t}+\delta
\end{align}

where:

$d$	- per capita dividends<br>
$y$	- per capita output<br>
$r^{k}$	- capital rental rate<br>
$k$	- per capita physical capital stock<br>
$w$	- real wage per hour<br>
$h$	- per capita hours (as fraction of total available time)<br>
$z$	- stochastic total factor productivity (TFP) level<br>
$\alpha$	- physical capital share in output<br>
$r$	- real interest rate<br>
$\delta$	- physical capital depreciation rate<br>

Rewritten problem:

\begin{align}
\max\quad d_{t}=z_{t}k_{t}^{\alpha}h_{t}^{1-\alpha}-\left(r_{t}+\delta\right)k_{t}-w_{t}h_{t}
\end{align}

First Order Conditions:
\begin{align}
\frac{\partial d_{t}}{\partial k_{t}}	=\alpha z_{t}k_{t}^{\alpha-1}h_{t}^{1-\alpha}-\left(r_{t}+\delta\right)=0\quad\,\to\quad r_{t}=\alpha z_{t}k_{t}^{\alpha-1}h_{t}^{1-\alpha}-\delta\\
\frac{\partial d_{t}}{\partial h_{t}}	=\left(1-\alpha\right)z_{t}k_{t}^{\alpha}h_{t}^{-\alpha}-w_{t}=0\quad\quad\to\quad w_{t}=\left(1-\alpha\right)z_{t}k_{t}^{\alpha}h_{t}^{-\alpha}
\end{align}

Alternative expressions for factor prices:
\begin{align}
r_{t}	=\alpha\frac{y_{t}}{k_{t}}-\delta\\
w_{t}	=\left(1-\alpha\right)\frac{y_{t}}{h_{t}}
\end{align}

Due to perfect competition and CRS economic profits equal zero:
\begin{align}
d_{t}=y_{t}-r_{t}^{k}k_{t}-w_{t}h_{t}=y_{t}-\alpha\frac{y_{t}}{k_{t}}\cdot k_{t}-\left(1-\alpha\right)\frac{y_{t}}{h_{t}}\cdot h_{t}=0
\end{align}


### General Equilibrium
Capital market clears:
\begin{align}
a_{t}=k_{t}
\end{align}

Households' budget constraint can be written as resource constraint:
\begin{align}
a_{t+1}+c_{t}	& =	\left(1+r_{t}\right)a_{t}+w_{t}h_{t}+d_{t}\\
k_{t+1}+c_{t}	& =	\left(1+\alpha\frac{y_{t}}{k_{t}}-\delta\right)k_{t}+\left(1-\alpha\right)\frac{y_{t}}{h_{t}}\cdot h_{t}+0\\
k_{t+1}+c_{t}	& =	\alpha y_{t}+\left(1-\delta\right)k_{t}+\left(1-\alpha\right)y_{t}\\
k_{t+1}+c_{t}	& =	y_{t}+\left(1-\delta\right)k_{t}
\end{align}

If we define investment as:
\begin{align}
i_{t}=k_{t+1}-\left(1-\delta\right)k_{t}
\end{align}

We can rewrite the resource constraint as the GDP accounting equation:
\begin{align}
y_{t}=c_{t}+i_{t}
\end{align}


### TFP Process
TFP evolves according to an AR(1) process:
\begin{align}
z_{t}=\left(1-\rho_{z}\right)+\rho_{z}\cdot z_{t-1}+\varepsilon_{t}
\end{align}

where $\rho_{z}<1$ regulates shock persistence and $\varepsilon$ is zero-mean white noise

It is often assumed that 
\begin{align}
\varepsilon\sim\mathcal{N}\left(0,\sigma_{z}^{2}\right)
\end{align}

In the absence of shocks
\begin{align}
z\to1\
\end{align}

### Full set of equilibrium conditions
System of 8 equations and 8 unknowns:
$\left\{ y,c,i,k,h,w,r,z\right\}$

\begin{align}
\text{Euler equation}	& :	\frac{1}{c_{t}}=\beta\mathrm{E}_{t}\left[\frac{1}{c_{t+1}}\left(1+r_{t+1}\right)\right]\\
\text{Consumption-hours choice}	& :	h_{t}=1-\phi\frac{c_{t}}{w_{t}}\\
\text{Production function}	& :	y_{t}=z_{t}k_{t}^{\alpha}h_{t}^{1-\alpha}\\
\text{Real interest rate}	& :	r_{t}=\alpha\frac{y_{t}}{k_{t}}-\delta\\
\text{Real hourly wage}	& :	w_{t}=\left(1-\alpha\right)\frac{y_{t}}{h_{t}}\\
\text{Investment}	& :	i_{t}=k_{t+1}-\left(1-\delta\right)k_{t}\\
\text{Output accounting}	& :	y_{t}=c_{t}+i_{t}\\
\text{TFP AR(1) process}	& :	z_{t}=\left(1-\rho_{z}\right)+\rho_{z}\cdot z_{t-1}+\varepsilon_{t}
\end{align}

### Solution Method

• Our model is a system of non-linear difference equations

• There exist no closed form solutions for the transitional dynamics except for few unrealistic cases

• We can solve easily an approximated version of the system

– (log-)linearize by hand around steady-state

– let Dynare compute n-th order Taylor expansion

• Solving the DSGE model involves transforming the forward looking system into a VAR (backward looking) system

– Many good methods: Blanchard-Kahn, Klein, Sims, etc.

• Computer software exists that does it for you, e.g. DynareDynare does that for you: stoch_simul;

• This is possible thanks to the Rational Expectations assumption

### Steady-State

Start with the Euler equation:<br>
$\frac{1}{c}=\beta\frac{1}{c}\left(1+r\right)\quad\to\quad r=\frac{1}{\beta}-1$

From the interest rate equation obtain the k/h ratio:<br> 
$r=\alpha k^{\alpha-1}h^{1-\alpha}-\delta\quad\to\quad\frac{k}{h}=\left(\frac{\alpha}{r+\delta}\right)^{\frac{1}{1-\alpha}}$

From the production function obtain the y/h ratio and then wage:<br>
$y=k^{\alpha}h^{1-\alpha}\quad\to\quad\frac{y}{h}=\left(\frac{k}{h}\right)^{\alpha}\quad\text{and}\quad w=\left(1-\alpha\right)\frac{y}{h}$

From investment and output accounting eqns. obtain the c/h ratio:<br>
$i=\delta k\quad\to\quad y=c+\delta k\quad\to\quad\frac{c}{h}=\frac{y}{h}-\delta\frac{k}{h}$

Get h from the consumption-hours choice. The rest follows from h:<br>
$h=1-\phi\frac{c}{w}\quad\to\quad1=\frac{1}{h}-\phi\frac{c}{h}\frac{1}{w}\quad\to\quad h=1/\left[1+\phi\frac{c}{h}\frac{1}{w}\right]$

### Log-linearized Equilibrium

Log-linearization means taking the log-deviation around a steady state value. Define the log-deviation of variable $x_{t}$ from its steady state $x^{*}$ as

\begin{align}
\hat{x}_{t}=\ln x_{t}-\ln x^{*}\approx\dfrac{x_{t}-x^{*}}{x^{*}}
\end{align}

The log-linearized system is given by 

\begin{align}
\text{Euler equation}	& :	\hat{c}_{t}=\mathrm{E}_{t}\hat{c}_{t+1}-\dfrac{1}{1+r^{*}}\mathrm{E}_{t}\left[\hat{r}_{t+1}\right]\\
\text{Consumption-hours choice} &	:	\hat{h}_{t}=\dfrac{\phi c^{*}}{w^{*}h^{*}}\left(\hat{w}_{t}-\hat{c}_{t}\right)\\
\text{Production function} &	:	\hat{y}_{t}=\hat{z}_{t}+\alpha\hat{k}_{t}+\left(1-\alpha\right)\hat{h}_{t}\\
\text{Real interest rate}	& :	\hat{r}_{t}=\alpha\dfrac{y^{*}}{k^{*}}\left(\hat{y}_{t}-\hat{k}_{t}\right)\\
\text{Real hourly wage}	& :	\hat{w}_{t}=\hat{y}_{t}-\hat{h}_{t}\\
\text{Investment}	& :	\hat{k}_{t+1}=\dfrac{i^{*}}{k^{*}}\hat{i}_{t}+\left(1-\delta\right)\hat{k}_{t}\\
\text{Output accounting}	& :	\hat{y}_{t}=\dfrac{c^{*}}{y^{*}}\hat{c}_{t}+\dfrac{i^{*}}{y^{*}}\hat{i}_{t}\\
\text{TFP AR(1) process}	& :	\hat{z}_{t}=\rho_{z}\hat{z}_{t-1}+\varepsilon_{t}
\end{align}

### Parameter values
• We need to specify parameter values tell Dynare the parameter values, e.g. $\alpha = 0.33$;

• There is a variety of approaches on how to obtain those values

• Two most widely used are:

– Calibration – picking parameter values to fit certain long-run (average) features of data. For example, we might want to pick the parameters so that the model's investment share in GDP matches the average share in the data

– Estimation – Dynare allows us to easily run a Bayesian estimation procedure on real data. It still needs as an input prior estimates of parameter values and their confidence intervals, which makes the calibration exercise very useful

• Most models in recent papers are estimated

• Today's toy model is calibrated

|Parameter | Value justification	|Mean
|:---|:---|:---|
$\alpha$ |	Capital income share of GDP	| 0.33	
$\beta$	| From average real interest rate	| 0.99	
$\delta$ |	From investment share of GDP	| 0.025	
$\phi$	| Work for 1/3 of time endowment	| 1.75	
$\rho_{z}$	| Coefficient in TFP AR(1) regression	| 0.97	
$\sigma_{z}$	| Error term in TFP AR(1) regression	| 0.007	


### Full set of equilibrium conditions
System of 8 equations and 8 unknowns:
$\left\{ y,c,i,k,h,w,r,z\right\}$

\begin{align}
\text{Euler equation}	& :	\frac{1}{c_{t}}=\beta\mathrm{E}_{t}\left[\frac{1}{c_{t+1}}\left(1+r_{t+1}\right)\right]\\
\text{Consumption-hours choice}	& :	h_{t}=1-\phi\frac{c_{t}}{w_{t}}\\
\text{Production function}	& :	y_{t}=z_{t}k_{t}^{\alpha}h_{t}^{1-\alpha}\\
\text{Real interest rate}	& :	r_{t}=\alpha\frac{y_{t}}{k_{t}}-\delta\\
\text{Real hourly wage}	& :	w_{t}=\left(1-\alpha\right)\frac{y_{t}}{h_{t}}\\
\text{Investment}	& :	i_{t}=k_{t+1}-\left(1-\delta\right)k_{t}\\
\text{Output accounting}	& :	y_{t}=c_{t}+i_{t}\\
\text{TFP AR(1) process}	& :	z_{t}=\left(1-\rho_{z}\right)+\rho_{z}\cdot z_{t-1}+\varepsilon_{t}
\end{align}

In [ ]:
from Dynare import *

In [ ]:
var = 'y c i k h w r z yh R'
varexo = 'e'

param_values = {'alpha':0.33, sy.symbols('beta'):0.99, 'delta':0.025, 'phi':1.7517, 'rho':rho_z}
# param_values = {'alpha':0.33, sy.symbols('beta'):0.99, 'delta':0.025, 'phi':1.7517, 'rho':0.9598499596182404}

model = ('-1/c + betta/c(+1)*(1+r(+1))',
         '-phi/(1-h) + w/c',
         '-y + z*k(-1)^alpha*h^(1-alpha)',
         '-r + alpha*y/k(-1) - delta',
         '-w + (1-alpha)*y/h',
         '-k + i + (1-delta)*k(-1)',
         '-y + c + i',
         '-log(z) + rho*log(z(-1)) + e',
         '-yh + y/h',
         '-R + 1+r')

initval = (1, 0.8, 0.2, 10, 0.33, 2, 0.01, 1, 3, 1.01)

In [ ]:
rbc = Dynare(var, varexo, param_values, model, initval)

rbc.system

In [ ]:
rbc.steady()

rbc.stoch_simul(irf=40)

In [ ]:
rbc.SimulatedMoments(hp_filter=1600, shocks_stderr=stderr_z)
# rbc.SimulatedMoments(hp_filter=1600, shocks_stderr=0.008016122502811389)

In [ ]:
print('Standard Deviations')
print(round(hp_cycles.std(),4))

print('')
print('Autocorrelations')
a = list(dta.columns.values[:10])
for i in range(len(a)):
    print(dta.columns.values[i], '  \t\t', round(hp_cycles[dta.columns.values[i]].autocorr(),4))

print('')
print('Correlations')
print(round(hp_cycles.corr(method='pearson'),4))

## Homework: role of Frisch elasticity

A key parameter that affects how volatile are hours worked over the business cycle is the so called [Frisch elasticity of labor supply](https://en.wikipedia.org/wiki/Frisch_elasticity_of_labor_supply).

It turns out, that our log-log utility function that we have used has a certain Frisch elasticity "baked in". For example, if the average hours worked are 1/3 of total time available, then it necessarily implies that the Frisch elasticity is equal to 2.

We might want to be able to change the value of this parameter. Therefore, we will use a slightly different utility function. The household will then solve the following problem:

\begin{align*}
\max\quad & U_{t}=\mathrm{E}_{t}\left[\sum_{i=0}^{\infty}\beta^{i}\left(\log c_{t+i}-\phi\frac{h_{t+i}^{1+\eta}}{1+\eta}\right)\right]\\
\text{subject to}\quad & c_{t}+a_{t+1}=w_{t}h_{t}+\left(1+r_{t}\right)a_{t}+d_{t}
\end{align*}

where the parameter $\eta$ (`eta`) is the inverse of the Frisch elasticity of labor supply.

**Task 1**: Derive the first order condition of the households and the optimality condition for the consumption-labor choice. Modify accordingly the code below.

In [ ]:
rho_z    = 0.9598499596182404
stderr_z = 0.008016122502811389

var = 'y c i k h w r z yh R'
varexo = 'e'

param_values_Frisch = {'alpha':0.33, sy.symbols('beta'):0.99, 'delta':0.025, 'phi':4.5511, 'rho':rho_z, 'eta':0.5}

model_Frisch = ('-1/c + betta/c(+1)*(1+r(+1))',
#          'Here goes the new consumption-labor choice equation',
         '-y + z*k(-1)^alpha*h^(1-alpha)',
         '-r + alpha*y/k(-1) - delta',
         '-w + (1-alpha)*y/h',
         '-k + i + (1-delta)*k(-1)',
         '-y + c + i',
         '-log(z) + rho*log(z(-1)) + e',
         '-yh + y/h',
         '-R + 1+r')

initval = (1, 0.8, 0.2, 10, 0.33, 2, 0.01, 1, 3, 1.01)

rbc_Frisch = Dynare(var, varexo, param_values_Frisch, model_Frisch, initval)

rbc_Frisch.system

**Task 2**: Verify that the above model produces exactly the same steady state and very similar simulated moments as the basic RBC model.

In [ ]:
# Your codes go here

**Task 3**: Use the alternative parametrization below. Under the new parametrization, compute the simulated moments of the model and compare the relative standard deviation of hours to output with the results we got for the basic RBC model.

In [ ]:
param_values_Frisch_alt = {'alpha':0.33, sy.symbols('beta'):0.99, 'delta':0.025, 'phi':2.7759, 'rho':rho_z, 'eta':0.05}

# Your codes go here